In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shusrith/imbd-dataset-tokenized")

print("Path to dataset files:", path)

100%|██████████| 18.9M/18.9M [00:01<00:00, 10.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shusrith/imbd-dataset-tokenized/versions/4


In [14]:
import pandas as pd
df = pd.read_csv("data.csv")
df

,review,sentiment
0,"[27, 4, 1, 75, 1944, 44, 1062, 11, 100, 143, 3...",1
1,"[119, 203, 3258, 68, 13, 36, 1581, 8, 12, 2198...",1
2,"[3, 384, 116, 356, 1, 1338, 2937, 6, 51, 17935...",1
3,"[9, 192, 10, 12, 3, 384, 94, 5, 1120, 57, 19, ...",1
4,"[667, 214, 3, 233, 113, 3, 116, 435, 3556, 120...",0
...,...,...
79534,"[9, 227, 3, 3510, 4325, 7, 0, 8342, 4602, 31, ...",0
79535,"[126, 109, 4, 82, 1, 1587, 4, 6509, 2263, 8903...",0
79536,"[140, 161, 5, 24, 5, 2964, 14, 1, 899, 912, 2,...",0
79537,"[700, 423, 2193, 17142, 1, 0, 11, 12204, 4212,...",0


In [15]:
import ast
df["review"] = df["review"].apply(lambda x: ast.literal_eval(x))
type(df.iloc[0, 0])

list

In [16]:
import torch.nn as nn
import torch


class Model(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)

        if h is None:
            h = torch.zeros(1, x.size(0), self.hidden_dim).to(
                "cuda"
            )  

        x, h = self.rnn(x, h)
        x = x[:, -1, :] 
        x = self.fc(x)
        return x, h.detach()  

In [17]:
model = Model(64, 128, 30522)

In [18]:
import torch.optim as optim

batch_size = 256
seq_len = 200
epochs = 10
learning_rate = 0.001

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
from torch.utils.data import DataLoader, TensorDataset
train = TensorDataset(
    torch.tensor(df["review"][:50000].tolist(), dtype=torch.long),
    torch.tensor(df["sentiment"][:50000].tolist(), dtype=torch.float32),
)
test = TensorDataset(
    torch.tensor(df["review"][50000:].tolist(), dtype=torch.long),
    torch.tensor(df["sentiment"][50000:].tolist(), dtype=torch.float32),
)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [20]:
vocab_size = 30522
hidden_dim = 128

In [ ]:
from tqdm import tqdm

model = model.to("cuda")

for epoch in range(epochs):
    model.train()
    train_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")

    total_train_loss = 0
    correct_train = 0
    total_train = 0

    for train_data, train_labels in train_tqdm:
        train_data, train_labels = train_data.to("cuda"), train_labels.to("cuda")

        optimizer.zero_grad()
        output, _ = model(train_data)
        output = output.squeeze()

        loss = criterion(output, train_labels)
        loss.backward()
        optimizer.step()

        predictions = (torch.sigmoid(output) > 0.5).float()  
        correct_train += (predictions == train_labels).sum().item()
        total_train += train_labels.size(0)

        total_train_loss += loss.item()
        train_tqdm.set_postfix(loss=loss.item())

    avg_train_loss = total_train_loss / len(train_loader)
    train_acc = correct_train / total_train

    # ------------------- Validation Loop -------------------
    model.eval()
    val_tqdm = tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]")

    total_val_loss = 0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for val_data, val_labels in val_tqdm:
            val_data, val_labels = val_data.to("cuda"), val_labels.to("cuda")

            output, _ = model(val_data)
            output = output.squeeze()

            loss = criterion(output, val_labels)
            total_val_loss += loss.item()

            predictions = (torch.sigmoid(output) > 0.5).float()
            correct_val += (predictions == val_labels).sum().item()
            total_val += val_labels.size(0)

            val_tqdm.set_postfix(loss=loss.item())

    avg_val_loss = total_val_loss / len(test_loader)
    val_acc = correct_val / total_val

    print(
        f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}"
    )

In [35]:
torch.nn.functional.sigmoid(output)

tensor([0.6199, 0.3544], device='cuda:0')

In [61]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
import numpy as np

# Set parameters
vocab_size = 10000  # Number of words to keep based on frequency
max_length = 200  # Max length of a review
embedding_dim = 128  # Embedding vector size
lstm_units = 64  # Number of LSTM units
batch_size = 64
epochs = 5

# Load IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to ensure uniform length
x_train = pad_sequences(x_train, maxlen=max_length, padding="post", truncating="post")
x_test = pad_sequences(x_test, maxlen=max_length, padding="post", truncating="post")

# Build the LSTM model
model = keras.Sequential(
    [
        keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        keras.layers.LSTM(lstm_units, return_sequences=True),
        keras.layers.LSTM(lstm_units),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(
            1, activation="sigmoid"
        ),  # Binary classification (positive/negative)
    ]
)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

2025-03-06 06:21:37.338819: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-06 06:21:37.433953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-06 06:21:37.472817: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-06 06:21:37.485991: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 06:21:37.559474: I tensorflow/core/platform/cpu_feature_guar

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/5


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1741222305.679446    6351 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-06 06:21:45.682150: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


391/391 ━━━━━━━━━━━━━━━━━━━━ 52s 127ms/step - accuracy: 0.5541 - loss: 0.6772 - val_accuracy: 0.5771 - val_loss: 0.6544
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 55s 140ms/step - accuracy: 0.5220 - loss: 0.6883 - val_accuracy: 0.5375 - val_loss: 0.6902
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 54s 137ms/step - accuracy: 0.5419 - loss: 0.6867 - val_accuracy: 0.5820 - val_loss: 0.6738
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 51s 130ms/step - accuracy: 0.6780 - loss: 0.5948 - val_accuracy: 0.7869 - val_loss: 0.4881
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 56s 143ms/step - accuracy: 0.8356 - loss: 0.3868 - val_accuracy: 0.8398 - val_loss: 0.3737
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - accuracy: 0.8413 - loss: 0.3729
Test Accuracy: 0.8398
